In [1]:
import pandas as pd

testdata = pd.read_csv('TestData.csv')

In [3]:
print(testdata.head())

    Program   TestArea Result             StartTime               EndTime
0      Meta  Diagnosis   PASS  12/5/2023 1:29:20 PM  12/5/2023 1:29:20 PM
1      Meta  Diagnosis   PASS  12/5/2023 1:29:21 PM  12/5/2023 1:29:21 PM
2      Meta  Diagnosis   PASS  12/5/2023 1:29:22 PM  12/5/2023 1:29:22 PM
3      Meta     H_FOVI   PASS  12/5/2023 1:29:22 PM  12/5/2023 1:29:22 PM
4  Spectrum   Spectrum   FAIL  12/5/2023 1:40:19 PM  12/5/2023 2:00:48 PM


In [ ]:
# Data Preprocessing
testdata['Program'] = testdata['Program'].astype('category')
testdata['TestArea'] = testdata['TestArea'].astype('category')
testdata['Result'] = testdata['Result'].astype('category')

testdata['StartTime'] = pd.to_datetime(testdata['StartTime'], format='%m/%d/%Y %I:%M:%S %p')
testdata['EndTime'] = pd.to_datetime(testdata['EndTime'], format='%m/%d/%Y %I:%M:%S %p')

print(testdata.info())

    Program   TestArea Result           StartTime             EndTime
0      Meta  Diagnosis   PASS 2023-12-05 13:29:20 2023-12-05 13:29:20
1      Meta  Diagnosis   PASS 2023-12-05 13:29:21 2023-12-05 13:29:21
2      Meta  Diagnosis   PASS 2023-12-05 13:29:22 2023-12-05 13:29:22
3      Meta     H_FOVI   PASS 2023-12-05 13:29:22 2023-12-05 13:29:22
4  Spectrum   Spectrum   FAIL 2023-12-05 13:40:19 2023-12-05 14:00:48
